(bayes-inference)=
# Bayesian Inference
In this section, we'll use Bayesian inference for the task of [*next word prediction*](https://arxiv.org/abs/2306.17184).

## Next Word Prediction
A common task that language models try to solve is next word prediction. Given a series of words in a sentence, can the language model predict the *next* word that should appear? Ideally, the model would use understanding of the prior words that appeared 



## Bayes' Formula
The traditional Bayes' formula, given events A and B, is:

$$
P(A|B) = \frac{P(B|A) * P(A)}{P(B)}
$$

Given a prior, $\theta$, which represents the probability distribution of a data object *before* it's observed, and a likelihood $y$, which represents the probability of falling under a specific category/class, we can generalize the above formula to compute the posterior as such:

$$
p(\theta|y) = \frac{p(y|\theta)*p(\theta)}{p(y)}
$$

where $p(\theta|y)$ represents the posterior probability: the probability after the evidence from our prior is considered.


## A Simple Application of Bayes for Next Word Prediction
Let's start simple. We'll take some text and tokenize it into words. I'll then create a matrix that shows the probability that a word appears given the word that comes directly before it. Again, this is a naive approach because I will *only* look at the word that comes immediately before. That makes the approach act more like a [Markovian transition matrix](https://www.jstor.org/stable/2584334), since it's just looking at the state immediately before to make a prediction, rather than all the events that led up to the prediction.

![](images/markov.png)

Markov chain analysis is a mathematical framework used to model and analyze systems that undergo transitions between different states over time. A Markov chain is a stochastic process where the future state depends only on the current state and not on the sequence of events that preceded it. In this case our "state" is the current word being observed. We will use the current state (word) to predict the next state (word).



In [1]:
text = '''
I am so excited to graduate from Georgia Tech! 
I have loved my time studying at Georgia Tech and cannot wait to graduate.
'''

print(text)



I am so excited to graduate from Georgia Tech! 
I have loved my time studying at Georgia Tech and cannot wait to graduate.



## Word Bi-gram Co-occurrence Matrix
To start, I'll tokenize the text into words and split it into bigrams. Using the bigrams, I'll create a co-occurence matrix, $P_{ij}$ where $i$ represents the $i^{\text{th}}$ word and $j$ represents the $j^{\text{th}}$ word. The value of $P_{ij}$ then is the probability that word $j$ occurs immediately after word $i$.

In [2]:
import nltk
import string
import pandas as pd
from sklearn.preprocessing import normalize
from tqdm import tqdm
from IPython.display import display, Math, Latex
from nltk.corpus import stopwords
import numpy as np

stop_words = stopwords.words('english')

def create_markov_matrix(text, order = 1):
    # Get a list of the tokenized words without punctuation
    tokens = [word.lower() for word in nltk.word_tokenize(text) if word not in string.punctuation]
    unique_tokens = list(dict.fromkeys(tokens))
    # print(unique_tokens)

    bigrams = list(nltk.bigrams(tokens))
    # print(bigrams)

    # Create a DataFrame where the rows are words and the columns are words
    df = pd.DataFrame(0, columns=unique_tokens, index=unique_tokens)

    # Loop through each of the bigrams (tuples), locate them in the DF, and add 1
    for i in bigrams:
        df.loc[i[0],i[1]] += 1

    # Convert the DataFrame from raw word counts to probabilities
    w_normalized = normalize(df, norm='l1', axis=1)

    if order > 1:
        w_normalized = np.linalg.matrix_power(w_normalized, order)
        
    df_normalized = pd.DataFrame(w_normalized, columns=unique_tokens, index=unique_tokens)

    return df_normalized

df_normalized = create_markov_matrix(text, order = 1)

display(df_normalized)

,i,am,so,excited,to,graduate,from,georgia,tech,have,loved,my,time,studying,at,and,can,not,wait
i,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
am,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
so,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
excited,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
to,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
graduate,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
from,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
georgia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tech,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
have,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the Markov transition matrix above we can see how, in the first row, given the word `I` (our current state), there are two words that could appear (our next state):

$$
P(\text{am } | \text{ I}) = .5 \\
P(\text{have } | \text{ I}) = .5
$$

It's important to note that these probabilities will **always** add up to 1; an important property of a [*row stochastic*](https://acme.byu.edu/00000179-af25-d5e1-a97b-bf6512fd0000/markov2020-pdf#:~:text=Thus%2C%20each%20of%20the%20columns,transition%20matrix%20sum%20to%201.&text=A%20transition%20matrix%20where%20the,stochastic%20(or%20left%20stochastic).) Markov chain.

In [29]:
def reshape_mm(mat):
    df_reshaped = mat.reset_index() \
                .rename(columns = {'index': 'first_word'}) \
                .melt(id_vars = 'first_word',
                    var_name = 'second_word',
                    value_name = 'p') \
                .sort_values('first_word')

    # Only keep actual words
    df_reshaped = df_reshaped[df_reshaped['first_word'].apply(lambda word: word.isalpha())]
    df_reshaped = df_reshaped[df_reshaped['second_word'].apply(lambda word: word.isalpha())]

    return df_reshaped.loc[(df_reshaped['p'] > 0)]


def find_next_word(mat, word, print_ = True):
    df_reshaped = reshape_mm(mat)

    # Filter our dataframe to the word of interest
    # Also make sure the second word is actually a word and not some punctuation that snuck in
    df_word = df_reshaped.loc[(df_reshaped['first_word'].str.lower() == word.lower()) & (df_reshaped['second_word'].str.isalpha())]

    # Sample using the probability column
    next_word_df = df_word.sample(1, weights = 'p')
    next_word = next_word_df['second_word'].iloc[0]
    p = next_word_df['p'].iloc[0]


    if print_:
        print(f'Selected `{next_word}` as the next word to follow `{word}` with probability {p:.3f}.')

    return next_word

find_next_word(df_normalized, word = 'I')


Selected `am` as the next word to follow `I` with probability 0.500.


'am'

## Broader Example
Let's apply this training mechanism to help write something even longer. For example, I'll load in the Georgia Tech student code of conduct and see if I can generate a new sentence based on it. I'll show a few sentence examples to better understand how the algorithm probabilistically builds sentences -- so that way each time it consults the Markov transition matrix, it selects a new word. I'll also try to define stop words so the sentence doesn't end with a stop word and flows a bit more naturally.

### Training Data
Our training data in this example is the text of interest. Using the training data, we'll build a Markov transition matrix. It's important to note that the more training data we have, the better our next word prediction will be.

In [30]:
new_text = open('honor_code.txt', 'r').read()

print(new_text[:100], '...')

# Build our Markov transition matrix
mm = create_markov_matrix(new_text)

print(f"The probability that we see the word `Tech` given that the word `Georgia` appears, based on our training dataset and Markov Transition matrix is {mm.loc['georgia', 'tech']:.3f}")

GEORGIA TECH HONOR CHALLENGE STATEMENT

I commit to uphold the ideals of honor and integrity by refu ...
The probability that we see the word `Tech` given that the word `Georgia` appears, based on our training dataset and Markov Transition matrix is 0.703


I've also included in the text above what should be a very common phrase -- Georgia Tech -- to show $P(\text{Tech } | \text{ Georgia}) = .703$. That is, given the word `Georgia` appears, there's a 70.3% chance that the word `Tech` appears directly after, regardless of any words that come before `Georgia`.

In [31]:
n_sent = 3
min_words = 8

def build_sents(mm, n_sent = 3, min_words = 8):
    for i, sent in enumerate(range(n_sent)):
        if i == 0:
            starting_word = 'students'
        else:
            starting_word = new_word

        sentence = [starting_word.title()]
        
        # Create boolean conditions for our while loop
        keep_going = True
        idx = 0

        while (keep_going) or (len(sentence) < min_words):
        # for idx, word in enumerate(range(n_words)):
            if idx == 0:
                new_word = starting_word

            new_word = find_next_word(mm, word = new_word, print_ = False)

            sentence.append(new_word)

            # If our final word is a stop word or we're over the minimum number of words
            # stop building our sentence and get rid of the stop word
            if new_word in stop_words:
                keep_going = False
                sentence.pop()
                new_word = find_next_word(mm, word = new_word, print_ = False)

            idx += 1

        print(f"{' '.join(sentence)}.")

build_sents(mm, n_sent = 3, min_words = 8)

Students according institute becomes aware academic credit notations.
Notations indicating rules conduct panel respondent drugs version.
Version allegation anyone community regarding georgia institute official.


And now we've managed to create brand new, never-before-seen sentences using a Markov transition matrix trained on the Georgia Tech honor code! While the sentences might not exactly make sense, we can begin to understand what the sentence is trying to say. But how can we actually make these human-readable? In the [next section](bayes-improvement), I'll explore various methods for improving the quality of these sentences.
